# Decision Tree Implementation

This implementation is part of the Intelligent Systems course at Monterrey Institute of Technology and Higher Education.

## Importing tools used for this implementation

In [29]:
import numpy as np 
import pandas as pd
from pandas.core.common import SettingWithCopyWarning
import warnings
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

## Get the data

In [30]:
# Read the file
df = pd.read_csv("cleveland.csv")
# Delete blank spaces in column names
df.columns = df.columns.str.replace(' ', '')
# Delete the rows where a missing value is found
df_complete_vals = df.loc[(df['MajorVessels'] != '?') 
                       & 
                       (df['thal'] != '?')]
# Split the dataset into X and y (inputs and target)
inputs = df_complete_vals.drop('num', axis='columns')
target = df_complete_vals['num']
# Split cathegorical data into multiple columns in order to have continuous data
# also called binary values. This process is called one hot encoding.
inputs_encoded = pd.get_dummies(inputs, columns=['ChestPainType', 
                                       'RestingElectrocardiographic', 
                                       'ExerciseSlope', 
                                       'thal'])
# Cathegorize values into two main targets: 0 - No heart disease and 1 to 4 - Heart disease
target_not_zero_index = target > 0
target[target_not_zero_index] = 1

totalinputs = inputs_encoded
totalinputs['num'] = target

totalinputs['thal_7'].unique()

array([0, 1], dtype=uint8)

## Node class

The node class describes the nodes of our decision tree. A tree contains two types of nodes:
- Decision nodes
    This nodes contain a condition. The condition is defined by the feature index and the threshold value for that particular feature. The left and right attributes are for accesing the left child and right child (they let us move from parent node to its child nodes). Information gain stores the information gain by the split denoted by this particular decision node.

- Leaf nodes
    The value is the majority class of the leaf node. It will help us to determine the class of a new data point if the data point ends up in this particular node.

In [31]:
class Node():
    # Constructor
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, information_gain=None, value=None):
        
        # Decision node
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.information_gain = information_gain
        
        # Leaf node
        self.value = value

## Decision Tree class

### Constructor

The first attribute is root, and it is used to traverse through the tree. The second and third attributes min_samples_split and max_depth are stopping conditions. For example, if in a particular node the number of samples becomes less than the minimum samples, then we won't split that node any further, and we will treat the node as a leaf node. If the depth of the tree reaches the maximum depth, we also won't split the nodes further.

### Build Tree

This is a function to build the binary tree using recursion. First it splits the features and targets into two separate variables which are x and y. Then we extract the number of samples and the number of features.

If the number of samples is greater or equal to minimum samples and the current depth is less or equal to maximum depth, we can split the tree. If these conditions are not satisfied, then we can't split the tree any further.

We use the get best split function to get the best split.

We check that the information gain corresponding to this split is greater than zero, so we don't split a node which is already pure (a node that consists of only one type of class).

We create the left subtree and the right subtree using recursion. First all left subtrees will be created and when a leaf node is reached, right subtrees will be created (the depth variable is increased in here).

After all subtrees are created, we return a node, which is a decision node. As it is a decision node we need to pass feature index, threshold, left subtree and right subtree connectors and the information gain.

Best split function is a dictionary that is returned by the get best split function.

We need to compute the leaf node using the function calculate leaf value. As it is a leaf node, we only need to pass one attribute that is the value.

### Get Best Split

This function returns a dictionary.

First we define an empty dictionary called best split and we initialize maximum information gain as negative infinity, because we want to maximize the information gain and to find that we have to use a number that is less than any other number.

#### First loop

We are going to loop through all the features and inside this loop we have to traverse through all the possible theshold values. The features are real numbers and there exists an infinite number of real numbers between any two real numbers, so it doesnt make sense to iterate through every possible real number, instead what we do is to traverse through every possible value of a feature that we have encountered in our data set, and np.unique function just returns the unique values of a particular feature, so that we can traverse through all the possible values of that feature.

#### Second loop

We split the data set based on the current feature index and the current threshold. At this point we have got the left data set and the right data set. We need to ensure that these are not empty, so once we know that we have something to work with, we extract the target values (denoted by y).

Now we need to compute the information gain, using the function called information gain. We are using gini index for calculating the information gain. Once we have got the current information gain, we need to check if this current information gain is greater than the max information gain. If it is greater than that, then we need to update the best split.

Once loops are completed, we just return the best split.

### Split

This function takes the dataset, the feature index and the theshold value and splits it into two parts. The first part will go to the left child and the second part will go to the right child. In the both left and right children we will send those data points that met our threshold condition.

### Information Gain

This function substracts the combined information of the child nodes from the parent node. Weights are the relative sizes of the child nodes with respect to the parent node.

### Entropy and Gini Index

These functions calculate Entropy and Gini index for a given array of target variables.


### Calculate leaf value

Calculates the value of the leaf node, which is just the majority class present in that particular node (it finds the most ocurring element in y).

### Fit 

In here we are concatenating the x and y to create our data set and then we are calling the build tree function. The root node will be returned by the build tree function and it is stored into self.root.

### Predict function

It takes a matrix of features and returns the corresponding predictions.

### Make prediction

It takes a node as a parameter. Initially we are just passing the root node. In the first conditional we check if the node is a leaf node, in case that it is a leaf node, it just returns the value. If the node is not a leaf node, then we extract the feature value of our new data point at the given feature index. Then we check if our feature value is less than or equal to the threshold. If it is true then we go through the left subtree, else we go through the right subtree.




In [32]:
class DecisionTreeClassifier():
    # Constructor
    def __init__(self, min_samples_split=2, max_depth=2):
        
        # Tree Root Initialization
        self.root = None
        
        # Stopping conditions
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        
    def build_tree(self, dataset, curr_depth=0):
        
        X, Y = dataset[:,:-1], dataset[:,-1]
        num_samples, num_features = np.shape(X)
        
        # Split until the conditions are not satisfied
        if num_samples>=self.min_samples_split and curr_depth<=self.max_depth:

            # Get the best split
            best_split = self.get_best_split(dataset, num_samples, num_features)

            # If the information gain is positive
            if best_split["information_gain"]>0:

                # Left recursion
                left_subtree = self.build_tree(best_split["dataset_left"], curr_depth+1)

                # Right recursion
                right_subtree = self.build_tree(best_split["dataset_right"], curr_depth+1)

                # Return the decision node
                return Node(best_split["feature_index"], best_split["threshold"], 
                            left_subtree, right_subtree, best_split["information_gain"])
        
        # Computing leaf node
        leaf_value = self.calculate_leaf_value(Y)

        # Return the leaf node
        return Node(value=leaf_value)
    
    def get_best_split(self, dataset, num_samples, num_features):
        
        # Dictionary that stores the best split
        best_split = {}
        max_info_gain = -float("inf")
        
        # loop over the features
        for feature_index in range(num_features):
            feature_values = dataset[:, feature_index]
            possible_thresholds = np.unique(feature_values)

            # loop over all the feature values present in the data
            for threshold in possible_thresholds:

                # Get current split
                dataset_left, dataset_right = self.split(dataset, feature_index, threshold)

                # Check if childs are not null
                if len(dataset_left)>0 and len(dataset_right)>0:
                    y, left_y, right_y = dataset[:, -1], dataset_left[:, -1], dataset_right[:, -1]

                    # Compute Information Gain
                    curr_info_gain = self.information_gain(y, left_y, right_y, "gini")

                    # Update best split
                    if curr_info_gain>max_info_gain:
                        best_split["feature_index"] = feature_index
                        best_split["threshold"] = threshold
                        best_split["dataset_left"] = dataset_left
                        best_split["dataset_right"] = dataset_right
                        best_split["information_gain"] = curr_info_gain
                        max_info_gain = curr_info_gain
                        
        # Return Best Split
        return best_split
    
    def split(self, dataset, feature_index, threshold):
        
        dataset_left = np.array([row for row in dataset if row[feature_index]<=threshold])
        dataset_right = np.array([row for row in dataset if row[feature_index]>threshold])
        return dataset_left, dataset_right
    
    def information_gain(self, parent, l_child, r_child, mode="entropy"):
        
        weight_l = len(l_child) / len(parent)
        weight_r = len(r_child) / len(parent)
        if mode=="gini":
            gain = self.gini_index(parent) - (weight_l*self.gini_index(l_child) + weight_r*self.gini_index(r_child))
        else:
            gain = self.entropy(parent) - (weight_l*self.entropy(l_child) + weight_r*self.entropy(r_child))
        return gain
    
    def entropy(self, y):
        
        class_labels = np.unique(y)
        entropy = 0
        for cls in class_labels:
            p_cls = len(y[y == cls]) / len(y)
            entropy += -p_cls * np.log2(p_cls)
        return entropy
    
    def gini_index(self, y):
        
        class_labels = np.unique(y)
        gini = 0
        for cls in class_labels:
            p_cls = len(y[y == cls]) / len(y)
            gini += p_cls**2
        return 1 - gini
        
    def calculate_leaf_value(self, Y):
        
        Y = list(Y)
        return max(Y, key=Y.count)
    
    def fit(self, X, Y):
        
        dataset = np.concatenate((X, Y), axis=1)
        self.root = self.build_tree(dataset)
    
    def predict(self, X):
        
        preditions = [self.make_prediction(x, self.root) for x in X]
        return preditions
    
    def make_prediction(self, x, tree):
        
        if tree.value!=None: return tree.value
        feature_val = x[tree.feature_index]
        if int(feature_val)<=int(tree.threshold):
            return self.make_prediction(x, tree.left)
        else:
            return self.make_prediction(x, tree.right)

## Train-Test Split

In [33]:
X = totalinputs.iloc[:, :-1].values
Y = totalinputs.iloc[:, -1].values.reshape(-1,1)
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.2, random_state=41)

## Fit Model

In [34]:
model = DecisionTreeClassifier(min_samples_split=2, max_depth=3)
model.fit(X_train,Y_train)

## Test Model

In [35]:
Y_pred = model.predict(X_test) 
from sklearn.metrics import accuracy_score
accuracy_score(Y_test, Y_pred)

0.7333333333333333

## References

1. https://www.sciencedirect.com/topics/nursing-and-health-professions/st-segment#:~:text=ST%20segment%20elevation%20or%20depression,is%20considered%20within%20normal%20limits.&text=ST%20segment%20elevation%20is%20more,present%20in%20the%20inferior%20leads.
2. https://www.nhs.uk/common-health-questions/lifestyle/what-is-blood-pressure/#:~:text=As%20a%20general%20guide%3A,be%2090%2F60mmHg%20or%20lower
3. https://mljar.com/blog/visualize-decision-tree/
4. http://rstudio-pubs-static.s3.amazonaws.com/24341_184a58191486470cab97acdbbfe78ed5.html
5. https://www.healthline.com/health/serum-cholesterol
6. https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1123032/
7. https://archive.ics.uci.edu/ml/datasets/Heart+Disease
8. http://rstudio-pubs-static.s3.amazonaws.com/24341_184a58191486470cab97acdbbfe78ed5.html
9. https://www.youtube.com/watch?v=jVh5NA9ERDA
